In [6]:
import pickle as pkl
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import to_categorical, Sequence
import os
import time
import matplotlib.pyplot as plt
from gensim.models import FastText, Doc2Vec

In [2]:
class_mapping = {}
with open('dbpedia/dbpedia_csv/classes.txt') as txt_file:
    line_num = 1
    for line in txt_file:
        class_mapping[line_num] = line.strip()
        line_num+=1
        
print(class_mapping)

{1: 'Company', 2: 'EducationalInstitution', 3: 'Artist', 4: 'Athlete', 5: 'OfficeHolder', 6: 'MeanOfTransportation', 7: 'Building', 8: 'NaturalPlace', 9: 'Village', 10: 'Animal', 11: 'Plant', 12: 'Album', 13: 'Film', 14: 'WrittenWork'}


In [ ]:
def plot_loss_acc(loss_acc):
    plt.plot(loss_acc['acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

    plt.plot(loss_acc['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    return

def plot_learning(lossAccPath):
    if os.path.exists(lossAccPath):
        loss_acc = pkl.load(open(lossAccPath, 'rb'))
    plot_loss_acc(loss_acc)
    return

In [ ]:
class Doc2Vec_FastText_SequenceDataGenerator(Sequence):
    def __init__(self, list_IDs, batch_size=32,
                 n_classes=14, shuffle=True):
        'Initialization'
        self.dim = (300,)
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate index of the batch
        indexes = self.indexes[index:(index+1)]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        
#         X = np.empty((self.batch_size, *self.dim))
        FT_array = np.empty((self.batch_size, *self.dim))
        DV_array = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype = int)
        for item in list_IDs_temp:
            Xarray = pkl.load(open('dbpedia/Training/'+item, 'rb'))
        for i, row in enumerate(Xarray[1]):
#             X[i,] = {'fastText_input':fastText.wv.__getitem__(row[-1]),'Doc2Vec_input': Doc2Vec_model.infer_vector(row)}
#             X.append([fastText.wv.__getitem__(row[-1]),Doc2Vec_model.infer_vector(row)])
            try:
                FT_array[i,] = fastText.wv[row[-1]]
            except KeyError:
                FT_array[i,] = np.zeros((300,))
            DV_array[i,] = Doc2Vec_model.infer_vector(row)
            y[i] = Xarray[0]-1

        return [FT_array,DV_array], to_categorical(y, num_classes=self.n_classes)

In [ ]:
fastText = FastText.load_fasttext_format('fasttext.bin')
# get vector, len 300 fastText.wv.__getitem__('Hello')
Doc2Vec_model = Doc2Vec.load("Doc2Vec.model")
Doc2Vec_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
partition, labels = pkl.load(open('dbpedia/Training/Metadata', 'rb'))
for key in labels.keys():
    labels[key] -= 1

In [ ]:
params = {'batch_size':32, 'n_classes': 14, 'shuffle': True}
training_generator = Doc2Vec_FastText_SequenceDataGenerator(partition['train'], **params)
validation_generator = Doc2Vec_FastText_SequenceDataGenerator(partition['validation'], **params)

### fastText 300 -> 16 train accuracy:
Epoch 1/5
24225/24225 [==============================] - 675s 28ms/step - loss: 1.0046 - acc: 0.6754 - val_loss: 0.8876 - val_acc: 0.7112
Epoch 2/5
24225/24225 [==============================] - 521s 21ms/step - loss: 0.8892 - acc: 0.7112 - val_loss: 0.8751 - val_acc: 0.7131
Epoch 3/5
24225/24225 [==============================] - 547s 23ms/step - loss: 0.8568 - acc: 0.7233 - val_loss: 0.8742 - val_acc: 0.7166
Epoch 4/5
24225/24225 [==============================] - 537s 22ms/step - loss: 0.8304 - acc: 0.7310 - val_loss: 0.7956 - val_acc: 0.7396
Epoch 5/5
24225/24225 [==============================] - 523s 22ms/step - loss: 0.8220 - acc: 0.7324 - val_loss: 0.8537 - val_acc: 0.7231

fastText_input = Input(shape = (300,), name = 'fastText_input')
fastText_output = Dense(16, activation = 'relu')(fastText_input)
Doc2Vec_input = Input(shape = (300,), name = 'Doc2Vec_input')
Doc2Vec_dense = Dense(128, activation = 'relu')(Doc2Vec_input)
Doc2Vec_output = Dense(128, activation = 'relu')(Doc2Vec_dense)
concatenated = keras.layers.concatenate([fastText_output, Doc2Vec_output])
predictions = Dense(14, activation = 'softmax')(concatenated)

model = Model(inputs = [fastText_input, Doc2Vec_input], outputs = predictions)
model.compile(optimizer ='Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.name = 'Doc2Vec_FastText'
loss_acc_path = model.name+'loss_acc'

### fastText 300 -> 32

In [ ]:
fastText_input = Input(shape = (300,), name = 'fastText_input')
fastText_output = Dense(32, activation = 'relu')(fastText_input)
Doc2Vec_input = Input(shape = (300,), name = 'Doc2Vec_input')
Doc2Vec_dense = Dense(128, activation = 'relu')(Doc2Vec_input)
Doc2Vec_output = Dense(128, activation = 'relu')(Doc2Vec_dense)
concatenated = keras.layers.concatenate([fastText_output, Doc2Vec_output])
predictions = Dense(14, activation = 'softmax')(concatenated)

model = Model(inputs = [fastText_input, Doc2Vec_input], outputs = predictions)
model.compile(optimizer ='Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.name = 'Doc2Vec_FastText32'
loss_acc_path = model.name+'loss_acc'

In [ ]:
history = model.fit_generator(generator = training_generator, epochs = 5, validation_data = validation_generator, verbose = 1)
loss_acc = {}
if os.path.exists(loss_acc_path):
    loss_acc = pkl.load(open(loss_acc_path, 'rb'))
    loss_acc['acc'] = loss_acc['acc'] + history.history['acc']
    loss_acc['loss'] = loss_acc['loss'] + history.history['loss']
else:
    loss_acc['acc'] = history.history['acc']
    loss_acc['loss'] = history.history['loss']
pkl.dump(loss_acc,open(loss_acc_path, 'wb'))
model.save(model.name+'.hdf5')

### fastText 300 -> 64

In [ ]:
fastText_input = Input(shape = (300,), name = 'fastText_input')
fastText_output = Dense(64, activation = 'relu')(fastText_input)
Doc2Vec_input = Input(shape = (300,), name = 'Doc2Vec_input')
Doc2Vec_dense = Dense(128, activation = 'relu')(Doc2Vec_input)
Doc2Vec_output = Dense(128, activation = 'relu')(Doc2Vec_dense)
concatenated = keras.layers.concatenate([fastText_output, Doc2Vec_output])
predictions = Dense(14, activation = 'softmax')(concatenated)

model = Model(inputs = [fastText_input, Doc2Vec_input], outputs = predictions)
model.compile(optimizer ='Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.name = 'Doc2Vec_FastText64'
loss_acc_path = model.name+'loss_acc'

In [ ]:
history = model.fit_generator(generator = training_generator, epochs = 5, validation_data = validation_generator, verbose = 1)
loss_acc = {}
if os.path.exists(loss_acc_path):
    loss_acc = pkl.load(open(loss_acc_path, 'rb'))
    loss_acc['acc'] = loss_acc['acc'] + history.history['acc']
    loss_acc['loss'] = loss_acc['loss'] + history.history['loss']
else:
    loss_acc['acc'] = history.history['acc']
    loss_acc['loss'] = history.history['loss']
pkl.dump(loss_acc,open(loss_acc_path, 'wb'))
model.save(model.name+'.hdf5')

### Testing

In [4]:
model = keras.models.load_model('Doc2Vec_FastText64.hdf5')
fastText = FastText.load_fasttext_format('fasttext.bin')
Doc2Vec_model = Doc2Vec.load('Doc2Vec.model')
Doc2Vec_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [26]:
start = time.time()
accuracies = {}
for label in os.listdir('dbpedia/Testing'):
    accuracies[label] = {}
    for sent_len in os.listdir('dbpedia/Testing/{}/'.format(label)):
        if int(sent_len) < 100:
            X_FT = np.empty((0,300))
            X_DV = np.empty((0,300))
            labels = []
            for file in os.listdir('dbpedia/Testing/{}/{}/'.format(label,sent_len)):
                sample = pkl.load(open('dbpedia/Testing/{}/{}/{}'.format(label,sent_len,file), 'rb'))
                labels.append(sample[0])
                try:
                    X_FT = np.vstack((X_FT,fastText.wv[sample[1][-1]]))
                except KeyError:
                    X_FT = np.vstack((X_FT,np.zeros((300,))))
                X_DV = np.vstack((X_DV, Doc2Vec_model.infer_vector(sample[1])))
            result = model.predict([X_FT,X_DV])
            result = np.argmax(result,axis = 1)+1
            accuracy = np.sum(result == labels)/len(labels)
            print('{}, sentence length: {}, Accuracy: {}'.format(label,sent_len,np.sum(result == labels)/len(labels)))
            accuracies[label][sent_len] = accuracy
            print('elapsed time: {}'.format((time.time()-start)/60))
    pkl.dump(accuracies, open('dbpedia/Doc2Vec_FastText_Accuracy', 'wb'))

Class1, sentence length: 1, Accuracy: 0.1458
elapsed time: 0.8674922386805216
Class1, sentence length: 10, Accuracy: 0.5881172219991971
elapsed time: 1.7749339580535888
Class1, sentence length: 11, Accuracy: 0.6233661773577317
elapsed time: 2.6157594164212545
Class1, sentence length: 12, Accuracy: 0.6387656313029447
elapsed time: 3.3889060775438944
Class1, sentence length: 13, Accuracy: 0.6504559270516718
elapsed time: 4.175719201564789
Class1, sentence length: 14, Accuracy: 0.652712863239179
elapsed time: 4.961594490210215
Class1, sentence length: 15, Accuracy: 0.6888390124464395
elapsed time: 5.711152136325836
Class1, sentence length: 16, Accuracy: 0.7008213552361396
elapsed time: 6.453095638751984
Class1, sentence length: 17, Accuracy: 0.7139026402640264
elapsed time: 7.1882950584093726
Class1, sentence length: 18, Accuracy: 0.7213557912247869
elapsed time: 7.910233076413473
Class1, sentence length: 19, Accuracy: 0.7379555927943025
elapsed time: 8.657510634263357
Class1, sentence le

Class1, sentence length: 93, Accuracy: 0.8292682926829268
elapsed time: 45.405997065703076
Class1, sentence length: 94, Accuracy: 0.8709677419354839
elapsed time: 45.41034103234609
Class1, sentence length: 95, Accuracy: 0.8214285714285714
elapsed time: 45.41268467903137
Class1, sentence length: 96, Accuracy: 0.8076923076923077
elapsed time: 45.414795776208244
Class1, sentence length: 97, Accuracy: 0.8333333333333334
elapsed time: 45.416878926753995
Class1, sentence length: 98, Accuracy: 0.85
elapsed time: 45.42028237183889
Class1, sentence length: 99, Accuracy: 0.8421052631578947
elapsed time: 45.42184477249781
Class10, sentence length: 1, Accuracy: 0.6544
elapsed time: 46.881628592809044
Class10, sentence length: 10, Accuracy: 0.7143723468768951
elapsed time: 48.06848592758179
Class10, sentence length: 11, Accuracy: 0.7128330282692699
elapsed time: 49.3319738705953
Class10, sentence length: 12, Accuracy: 0.7466637497265368
elapsed time: 50.250942778587344
Class10, sentence length: 13,

Class10, sentence length: 87, Accuracy: 0.856
elapsed time: 76.15461974143982
Class10, sentence length: 88, Accuracy: 0.839622641509434
elapsed time: 76.16361314853033
Class10, sentence length: 89, Accuracy: 0.7840909090909091
elapsed time: 76.17129846016566
Class10, sentence length: 9, Accuracy: 0.6598311218335344
elapsed time: 77.2507178068161
Class10, sentence length: 90, Accuracy: 0.7922077922077922
elapsed time: 77.26027356783548
Class10, sentence length: 91, Accuracy: 0.9090909090909091
elapsed time: 77.26598205963771
Class10, sentence length: 92, Accuracy: 0.8392857142857143
elapsed time: 77.27102247476577
Class10, sentence length: 93, Accuracy: 0.86
elapsed time: 77.27521441380183
Class10, sentence length: 94, Accuracy: 0.875
elapsed time: 77.2787461121877
Class10, sentence length: 95, Accuracy: 0.9705882352941176
elapsed time: 77.28171911239625
Class10, sentence length: 96, Accuracy: 0.9032258064516129
elapsed time: 77.28455736637116
Class10, sentence length: 97, Accuracy: 0.8

Class11, sentence length: 8, Accuracy: 0.7473937449879712
elapsed time: 113.58227065006892
Class11, sentence length: 80, Accuracy: 0.9643410852713178
elapsed time: 113.64562066793442
Class11, sentence length: 81, Accuracy: 0.9457092819614711
elapsed time: 113.69951261679331
Class11, sentence length: 82, Accuracy: 0.9368836291913215
elapsed time: 113.74608912467957
Class11, sentence length: 83, Accuracy: 0.9548532731376975
elapsed time: 113.78491228818893
Class11, sentence length: 84, Accuracy: 0.9532467532467532
elapsed time: 113.82069346904754
Class11, sentence length: 85, Accuracy: 0.9476923076923077
elapsed time: 113.8487003684044
Class11, sentence length: 86, Accuracy: 0.9398496240601504
elapsed time: 113.87208395799001
Class11, sentence length: 87, Accuracy: 0.9579439252336449
elapsed time: 113.8904898762703
Class11, sentence length: 88, Accuracy: 0.9777777777777777
elapsed time: 113.9062025944392
Class11, sentence length: 89, Accuracy: 0.9415584415584416
elapsed time: 113.9204570

Class12, sentence length: 72, Accuracy: 0.9570051890289103
elapsed time: 159.0250218351682
Class12, sentence length: 73, Accuracy: 0.9611197511664075
elapsed time: 159.1819777448972
Class12, sentence length: 74, Accuracy: 0.9593165174938975
elapsed time: 159.32933063904446
Class12, sentence length: 75, Accuracy: 0.9639794168096055
elapsed time: 159.46801055669783
Class12, sentence length: 76, Accuracy: 0.957619477006312
elapsed time: 159.59780603647232
Class12, sentence length: 77, Accuracy: 0.9572243346007605
elapsed time: 159.7189509987831
Class12, sentence length: 78, Accuracy: 0.9566968781470292
elapsed time: 159.82882202068964
Class12, sentence length: 79, Accuracy: 0.9598698481561823
elapsed time: 159.93101763327917
Class12, sentence length: 8, Accuracy: 0.7250900360144058
elapsed time: 161.02952831188838
Class12, sentence length: 80, Accuracy: 0.9574468085106383
elapsed time: 161.11807096004486
Class12, sentence length: 81, Accuracy: 0.9590268886043534
elapsed time: 161.19747114

...

Class5, sentence length: 32, Accuracy: 0.8209756097560975
elapsed time: 439.0951041976611
Class5, sentence length: 33, Accuracy: 0.8202330771138111
elapsed time: 439.8845457394918
Class5, sentence length: 34, Accuracy: 0.8197424892703863
elapsed time: 440.61810454527534
Class5, sentence length: 35, Accuracy: 0.8286451612903226
elapsed time: 441.3598043640455
Class5, sentence length: 36, Accuracy: 0.8405263157894737
elapsed time: 442.27609423398974
Class5, sentence length: 37, Accuracy: 0.8293469497446923
elapsed time: 442.972594443957
Class5, sentence length: 38, Accuracy: 0.8353424657534246
elapsed time: 443.6749325911204
Class5, sentence length: 39, Accuracy: 0.8491091314031181
elapsed time: 444.35181862513224
Class5, sentence length: 4, Accuracy: 0.23424684936987397
elapsed time: 445.45849358240764
Class5, sentence length: 40, Accuracy: 0.8404768663071246
elapsed time: 446.0951494614283
Class5, sentence length: 41, Accuracy: 0.8482220294882914
elapsed time: 446.6847177108129
Class5,

Class6, sentence length: 26, Accuracy: 0.8553333333333333
elapsed time: 485.97629629770915
Class6, sentence length: 27, Accuracy: 0.862533692722372
elapsed time: 486.8968030889829
Class6, sentence length: 28, Accuracy: 0.8697431234371448
elapsed time: 487.83173662026724
Class6, sentence length: 29, Accuracy: 0.8707903780068729
elapsed time: 488.7356362938881
Class6, sentence length: 3, Accuracy: 0.3724
elapsed time: 489.888070889314
Class6, sentence length: 30, Accuracy: 0.8792943361188487
elapsed time: 490.8732716917992
Class6, sentence length: 31, Accuracy: 0.8795860771401693
elapsed time: 491.759189804395
Class6, sentence length: 32, Accuracy: 0.8781822507732572
elapsed time: 492.57512766917546
Class6, sentence length: 33, Accuracy: 0.8871511068334937
elapsed time: 493.4042007565498
Class6, sentence length: 34, Accuracy: 0.8844368433911556
elapsed time: 494.24123961925505
Class6, sentence length: 35, Accuracy: 0.8907646474677259
elapsed time: 495.0182065606117
Class6, sentence lengt

Class7, sentence length: 20, Accuracy: 0.6212404065546567
elapsed time: 537.0658408482869
Class7, sentence length: 21, Accuracy: 0.6299311208515967
elapsed time: 538.0428627371788
Class7, sentence length: 22, Accuracy: 0.6446263643996641
elapsed time: 539.0132045388222
Class7, sentence length: 23, Accuracy: 0.6501057082452432
elapsed time: 539.9745647788047
Class7, sentence length: 24, Accuracy: 0.667590027700831
elapsed time: 540.9226003050804
Class7, sentence length: 25, Accuracy: 0.6645189003436426
elapsed time: 541.8977462569873
Class7, sentence length: 26, Accuracy: 0.6731601731601732
elapsed time: 542.8721399346988
Class7, sentence length: 27, Accuracy: 0.6728665207877462
elapsed time: 543.8730869611104
Class7, sentence length: 28, Accuracy: 0.684512007050011
elapsed time: 544.8338668028514
Class7, sentence length: 29, Accuracy: 0.6875555555555556
elapsed time: 545.7308227181435
Class7, sentence length: 3, Accuracy: 0.4192
elapsed time: 546.8091297189395
Class7, sentence length: 

Class8, sentence length: 15, Accuracy: 0.7219293621507644
elapsed time: 588.2432292342186
Class8, sentence length: 16, Accuracy: 0.7260681449432126
elapsed time: 588.9223435084025
Class8, sentence length: 17, Accuracy: 0.7323399558498896
elapsed time: 589.596939321359
Class8, sentence length: 18, Accuracy: 0.7506994963626189
elapsed time: 590.2023569464684
Class8, sentence length: 19, Accuracy: 0.739031339031339
elapsed time: 590.8281536459923
Class8, sentence length: 2, Accuracy: 0.2634
elapsed time: 592.1242518226305
Class8, sentence length: 20, Accuracy: 0.7588693394865879
elapsed time: 592.7741741458575
Class8, sentence length: 21, Accuracy: 0.7662111078801978
elapsed time: 593.3840622504553
Class8, sentence length: 22, Accuracy: 0.7714705882352941
elapsed time: 593.987418226401
Class8, sentence length: 23, Accuracy: 0.7819101457899434
elapsed time: 594.5755760351817
Class8, sentence length: 24, Accuracy: 0.7875491084919916
elapsed time: 595.1316164294878
Class8, sentence length: 2

Class9, sentence length: 1, Accuracy: 0.5466
elapsed time: 622.7006516098976
Class9, sentence length: 10, Accuracy: 0.8692416012874673
elapsed time: 623.9319623152415
Class9, sentence length: 11, Accuracy: 0.864881192106323
elapsed time: 625.0786417921385
Class9, sentence length: 12, Accuracy: 0.9226250759570589
elapsed time: 626.2156420429548
Class9, sentence length: 13, Accuracy: 0.9272357723577236
elapsed time: 627.2962035298348
Class9, sentence length: 14, Accuracy: 0.9215810610864511
elapsed time: 628.3657985687256
Class9, sentence length: 15, Accuracy: 0.9431208413822709
elapsed time: 629.3788612524669
Class9, sentence length: 16, Accuracy: 0.9504205400619743
elapsed time: 630.3289287805558
Class9, sentence length: 17, Accuracy: 0.9497870432638422
elapsed time: 631.3070798277855
Class9, sentence length: 18, Accuracy: 0.959419632736341
elapsed time: 632.2616864959399
Class9, sentence length: 19, Accuracy: 0.9655889145496536
elapsed time: 633.1679320931435
Class9, sentence length: 